In [ ]:
import glob

listOfFileNames = []
listOfClassNames = []
listOfDataPerClass = []
files = []
classNames = []

classNames = glob.glob("/Users/user/Desktop/Thesis/placeholder/*")

for c in classNames:
    className = c.split("\\")[1]
    listOfClassNames.append(c.split("\\")[1])
    files = glob.glob("/Users/user/Desktop/Thesis/placeholder/"+className+"/*.csv")
    listOfFileNames = []
    for f in files:
        listOfFileNames.append(f.split("\\")[1])
    listOfDataPerClass.append(listOfFileNames)

    
#listOfClassNames
#listOfDataPerClass[0]
print("Done listing the class and file names")

In [ ]:
#len(listOfDataPerClass)
#listOfDataPerClass[3]

import pandas as pd


listOfDataFramesPerClass = {}
dataFrameDictionary = {}
dataFrames = []

for c, d in zip(listOfClassNames, listOfDataPerClass):
    print("----------------- Word " + c + "-----------------")
    emgDataFrames = {}
    emgDataFrames['right'] = []
    emgDataFrames['left'] = []
    
    accDataFrames = {}
    accDataFrames['right'] = []
    accDataFrames['left'] = []

    gyroDataFrames = {}
    gyroDataFrames['right'] = []
    gyroDataFrames['left'] = []

    dataFrameDictionary = {}
    className = c
    emgleftctr = 0
    emgrightctr = 0
    accleftctr = 0
    accrightctr = 0
    gyroleftctr = 0
    gyrorightctr = 0
    for fileName in d:
        fileLocation = "/Users/user/Desktop/Thesis/placeholder/"+className+"/"+fileName + ""
        df = pd.read_csv(fileLocation)
        
        if "emg" in fileName:   
            if "-0-" in fileName:
                emgDataFrames['right'].append(df)
                emgleftctr+=1
            else:
                emgDataFrames['left'].append(df)
                emgrightctr+=1
        elif ("accelerometer") in fileName:
            if "-0-" in fileName:
                accDataFrames['right'].append(df)
                accleftctr+=1
            else:
                accDataFrames['left'].append(df)
                accrightctr+=1
        elif ("gyro") in fileName:
            if "-0-" in fileName:
                gyroDataFrames['right'].append(df)
                gyroleftctr+=1
            else:
                gyroDataFrames['left'].append(df)
                gyrorightctr+=1
    
    dataFrameDictionary["emg"] = emgDataFrames
    dataFrameDictionary["acc"] = accDataFrames
    dataFrameDictionary["gyro"] = gyroDataFrames

    listOfDataFramesPerClass[className] = dataFrameDictionary
    print("Total of " + str(emgleftctr) + " left arm emg files and " + str(emgrightctr) + " right arm emg files.")
    print("Total of " + str(accleftctr) + " left arm acc files and " + str(accrightctr) + " right arm acc files.")
    print("Total of " + str(gyroleftctr) + " left arm gyro files and " + str(gyrorightctr) + " right arm gyro files.")


print("Done converting to dataframes!")


## For EMG Values

In [ ]:
for a in listOfDataFramesPerClass[k]['emg']:
    print(a)

# Trimming

In [ ]:
import pandas as pd

listOfTrimmedDataFramesPerArm = {}
listOfTrimmedDataFramesPerClass = {}

for k in listOfDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    trimmedDataFramesPerClass = {}
    trimmedDataFramesPerClass['left'] = []
    trimmedDataFramesPerClass['right'] = []
    
    dataFrameDictionary = {}

    for a in listOfDataFramesPerClass[k]['emg']:
        for d in listOfDataFramesPerClass[k]['emg'][a]:
            keep_col = ['emg1','emg2','emg3','emg4','emg5','emg6','emg7','emg8']
            new_df = d[keep_col]
            new_df = new_df[:-49]
            trimmed_df = new_df.iloc[49:]
            if len(trimmed_df) < 100:
                print(str(ctr) + " " + k)
            trimmedDataFramesPerClass[a].append(trimmed_df)
            ctr += 1
        
    print("Done Trimming " + str(ctr) + " files")
    dataFrameDictionary["emg"] = trimmedDataFramesPerClass
    listOfTrimmedDataFramesPerClass[k] = dataFrameDictionary

# Normalization

In [ ]:
from sklearn import preprocessing

listOfNormalizedDataFramesPerClass = {}

for k in listOfTrimmedDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    NormalizedDataFramesPerClass = {}
    NormalizedDataFramesPerClass['right'] = []
    NormalizedDataFramesPerClass['left'] = []
    
    dataFrameDictionary = {}

    for a in listOfDataFramesPerClass[k]['emg']:
        ctr = 0
        for d in listOfDataFramesPerClass[k]['emg'][a]:
            #print(str(len(d.index)) + " " + str(ctr))
            # Create x, where x the 'scores' column's values as floats
            x = d.values.astype(float)

            # Create a minimum and maximum processor object
            min_max_scaler = preprocessing.MinMaxScaler()

            # Create an object to transform the data to fit minmax processor
            x_scaled = min_max_scaler.fit_transform(x)

            # Run the normalizer on the dataframe
            df_normalized = pd.DataFrame(x_scaled)
            #df_normalized
            NormalizedDataFramesPerClass[a].append(df_normalized)
            ctr += 1
        
        print("Done Normalizing " + str(ctr) + a + " arm files")
    dataFrameDictionary["emg"] = NormalizedDataFramesPerClass
    listOfNormalizedDataFramesPerClass[k] = dataFrameDictionary  
    
print("Data Normalization done!!")



In [ ]:
"left" + " emg"

In [ ]:
import pandas as pd
from statsmodels import robust
import numpy as np

listOfVarianceDataFramePerClass = {}
listOfVarianceDataFramePerClass['right'] = []
listOfVarianceDataFramePerClass['left'] = []

listOfMavDataFramePerClass = {}
listOfMavDataFramePerClass['right'] = []
listOfMavDataFramePerClass['left'] = []

listOfMeanDataFramePerClass = {}
listOfMeanDataFramePerClass['right'] = []
listOfMeanDataFramePerClass['left'] = []

listOfSdDataFramePerClass = {}
listOfSdDataFramePerClass['right'] = []
listOfSdDataFramePerClass['left'] = []

listOfRmsDataFramePerClass = {}
listOfRmsDataFramePerClass['right'] = []
listOfRmsDataFramePerClass['left'] = []

for k in listOfNormalizedDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0

    for a in listOfNormalizedDataFramesPerClass[k]['emg']:
        ctr = 0
        arm = a + " "

        varianceDataFrame = pd.DataFrame(columns = [(arm + 'variance of EMG1'), (arm + 'variance of EMG2'), (arm + 'variance of EMG3'), (arm + 'variance of EMG4'),
                                 (arm + 'variance of EMG5'), (arm + 'variance of EMG6'), (arm + 'variance of EMG7'), (arm + 'variance of EMG8')])
        mavDataFrame = pd.DataFrame(columns = [(arm + 'Mav of EMG1'), (arm + 'Mav of EMG2'), (arm + 'Mav of EMG3'), (arm + 'Mav of EMG4'),
                                     (arm + 'Mav of EMG5'), (arm + 'Mav of EMG6'), (arm + 'Mav of EMG7'), (arm + 'Mav of EMG8')])
        meanDataFrame = pd.DataFrame(columns = [(arm + 'Mean of EMG1'), (arm + 'Mean of EMG2'), (arm + 'Mean of EMG3'), (arm + 'Mean of EMG4'),
                                     (arm + 'Mean of EMG5'), (arm + 'Mean of EMG6'), (arm + 'Mean of EMG7'), (arm + 'Mean of EMG8')])
        sdDataFrame = pd.DataFrame(columns = [(arm + 'Sd of EMG1'), (arm + 'Sd of EMG2'), (arm + 'Sd of EMG3'), (arm + 'Sd of EMG4'),
                                     (arm + 'Sd of EMG5'), (arm + 'Sd of EMG6'), (arm + 'Sd of EMG7'), (arm + 'Sd of EMG8')])
        rmsDataFrame = pd.DataFrame(columns = [(arm + 'Rms of EMG1'), (arm + 'Rms of EMG2'), (arm + 'Rms of EMG3'), (arm + 'Rms of EMG4'),
                                     (arm + 'Rms of EMG5'), (arm + 'Rms of EMG6'), (arm + 'Rms of EMG7'), (arm + 'Rms of EMG8')])
        for df_normalized in listOfNormalizedDataFramesPerClass[k]['emg'][a]:
            varianceDataFrame.loc[ctr] = [df_normalized.var()[0], df_normalized.var()[1], df_normalized.var()[2], df_normalized.var()[3], df_normalized.var()[4], df_normalized.var()[5], df_normalized.var()[6], df_normalized.var()[7]]
            mavDataFrame.loc[ctr] = [robust.mad(df_normalized)[0], robust.mad(df_normalized)[1], robust.mad(df_normalized)[2], robust.mad(df_normalized)[3], robust.mad(df_normalized)[4], robust.mad(df_normalized)[5], robust.mad(df_normalized)[6], robust.mad(df_normalized)[7]]
            meanDataFrame.loc[ctr] = [np.mean(df_normalized)[0], np.mean(df_normalized)[1], np.mean(df_normalized)[2], np.mean(df_normalized)[3], np.mean(df_normalized)[4], np.mean(df_normalized)[5], np.mean(df_normalized)[6], np.mean(df_normalized)[7]]
            sdDataFrame.loc[ctr] = [np.std(df_normalized)[0], np.std(df_normalized)[1], np.std(df_normalized)[2], np.std(df_normalized)[3], np.std(df_normalized)[4], np.std(df_normalized)[5], np.std(df_normalized)[6], np.std(df_normalized)[7]]
            rmsDataFrame.loc[ctr] = [np.sqrt(np.mean(df_normalized[0].values**2)), np.sqrt(np.mean(df_normalized[1].values**2)), np.sqrt(np.mean(df_normalized[2].values**2)), np.sqrt(np.mean(df_normalized[3].values**2)), np.sqrt(np.mean(df_normalized[4].values**2)), np.sqrt(np.mean(df_normalized[5].values**2)), robust.mad(df_normalized)[6], np.sqrt(np.mean(df_normalized[7].values**2))]
            ctr +=1 
            
        print(len(varianceDataFrame))
        listOfVarianceDataFramePerClass[a].append(varianceDataFrame)
        listOfMavDataFramePerClass[a].append(mavDataFrame)
        listOfMeanDataFramePerClass[a].append(meanDataFrame)
        listOfSdDataFramePerClass[a].append(sdDataFrame)
        listOfRmsDataFramePerClass[a].append(rmsDataFrame)
    
        print("Done computing the var, mav, mean , sd, and rms of " + str(ctr) + a + " arm files")

print("Done computing emg features!")

In [ ]:
import pandas as pd
import numpy as np

listOfMeanFreqDataFramePerClass = {}
listOfMeanFreqDataFramePerClass['right'] = []
listOfMeanFreqDataFramePerClass['left'] = []

listOfMedFreqDataFramePerClass = {}
listOfMedFreqDataFramePerClass['right'] = []
listOfMedFreqDataFramePerClass['left'] = []

for k in listOfNormalizedDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    
    
    for a in listOfNormalizedDataFramesPerClass[k]['emg']:
        ctr = 0
        arm = a + " "

        meanFreqDataFrame = pd.DataFrame(columns = [(arm + 'MeanFreq of EMG1'), (arm + 'MeanFreq of EMG2'), (arm + 'MeanFreq of EMG3'), (arm + 'MeanFreq of EMG4'),
                                 (arm + 'MeanFreq of EMG5'), (arm + 'MeanFreq of EMG6'), (arm + 'MeanFreq of EMG7'), (arm + 'MeanFreq of EMG8')]) 

        medFreqDataFrame = pd.DataFrame(columns = [(arm + 'MedianFreq of EMG1'), (arm + 'MedianFreq of EMG2'), (arm + 'MedianFreq of EMG3'), (arm + 'MedianFreq of EMG4'),
                                     (arm + 'MedianFreq of EMG5'), (arm + 'MedianFreq of EMG6'), (arm + 'MedianFreq of EMG7'), (arm + 'MedianFreq of EMG8')])

        for df_normalized in listOfNormalizedDataFramesPerClass[k]['emg'][a]:
            meanFreq = []
            medFreq = []

            for i in range(0, 8):
                data = df_normalized[i].values
                ps = np.abs(np.fft.fft(data))**2

                freqs = np.fft.fftfreq(data.size, 1/100)

                sumProd = 0
                sumPs = 0

                for f, p in zip(freqs,ps):
                    sumProd += f*p
                    sumPs += p

                meanFreq.append(sumProd/sumPs)
                medFreq.append(sumPs/2)

            meanFreqDataFrame.loc[ctr] = [meanFreq[0], meanFreq[1], meanFreq[2], meanFreq[3], meanFreq[4], meanFreq[5], meanFreq[6], meanFreq[7]]
            medFreqDataFrame.loc[ctr] = [medFreq[0], medFreq[1], medFreq[2], medFreq[3], medFreq[4], medFreq[5], medFreq[6], medFreq[7]]

            ctr+=1
        listOfMeanFreqDataFramePerClass[a].append(meanFreqDataFrame)
        listOfMedFreqDataFramePerClass[a].append(medFreqDataFrame)
        print("Done computing Mean and Median Frequency of " + str(ctr) + a + " arm files")

print("Done computing emg features!")

Label Class

In [ ]:
#print(listOfMavDataFramePerClass[0])
#print(listOfRmsDataFramePerClass[0])
ctr = 0

listOfLabeledDataPerClass = {}
listOfLabeledDataPerClass['right'] = {}
listOfLabeledDataPerClass['left'] = {}


for a in listOfNormalizedDataFramesPerClass[k]['emg']:

    dataFramesPerFeature = {}
    ctr = 0
    for k in listOfTrimmedDataFramesPerClass.keys():
        dataFramesPerFeature['variance'] = listOfVarianceDataFramePerClass[a][ctr]
        dataFramesPerFeature['mav'] = listOfMavDataFramePerClass[a][ctr]
        dataFramesPerFeature['rms'] = listOfRmsDataFramePerClass[a][ctr]
        dataFramesPerFeature['sd'] = listOfSdDataFramePerClass[a][ctr]
        dataFramesPerFeature['mean'] = listOfMeanDataFramePerClass[a][ctr]
        dataFramesPerFeature['medFreq'] = listOfMedFreqDataFramePerClass[a][ctr]
        dataFramesPerFeature['meanFreq'] = listOfMeanFreqDataFramePerClass[a][ctr]
    

        for k2 in dataFramesPerFeature.keys():
            if k2 != 'variance':
                for i in dataFramesPerFeature[k2].columns:
                    dataFramesPerFeature['variance'][i] = dataFramesPerFeature[k2][i]
        
        dataFramesPerFeature['variance']['Class'] = k
        listOfLabeledDataPerClass[a][k] = dataFramesPerFeature['variance']
        print("Done labeling emg data of " + k +" "+ a + " arm files")
        ctr += 1
 

Data Consolidation (save to csv)

In [ ]:

for a in listOfNormalizedDataFramesPerClass[k]['emg']:
    newDf = listOfLabeledDataPerClass[a]['ano']
    for k in listOfTrimmedDataFramesPerClass.keys():
        if k != 'ano':
            print(k)
            newDf= newDf.append(listOfLabeledDataPerClass[a][k])
    newDf.to_csv('/Users/user/Desktop/'+ a + 'ArmConsolidatedEMG.csv', encoding='utf-8', index=False)

## For IMU values

Trimming (For Acceleration and Gyroscope)

In [ ]:
listOfDataFramesPerClass['isa']['gyro']['right']

In [ ]:
import pandas as pd

listOfTrimmedAccDataFramesPerClass = {}

for k in listOfDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    trimmedDataFramesPerClass = {}
    trimmedDataFramesPerClass['left'] = []
    trimmedDataFramesPerClass['right'] = []
    dataFrameDictionary = {}
    
    for a in listOfDataFramesPerClass[k]['emg']:
        for d in listOfDataFramesPerClass[k]['acc'][a]:
            keep_col = ['x','y','z']
            new_df = d[keep_col]
            new_df = new_df[:-24]
            trimmed_df = new_df.iloc[24:]
            if len(trimmed_df) < 50:
                print(str(ctr) + " " + str(len(trimmed_df)))
            trimmedDataFramesPerClass[a].append(trimmed_df)
            ctr += 1
        
    print("Done Trimming " + str(ctr) + " files")
    dataFrameDictionary["acc"] = trimmedDataFramesPerClass
    listOfTrimmedAccDataFramesPerClass[k] = dataFrameDictionary
    
print("Done Trimming Acc data!")

In [ ]:
import pandas as pd

listOfTrimmedGyroDataFramesPerClass = {}

for k in listOfDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    trimmedDataFramesPerClass = {}
    trimmedDataFramesPerClass['left'] = []
    trimmedDataFramesPerClass['right'] = []
    dataFrameDictionary = {}
    
    for a in listOfDataFramesPerClass[k]['gyro']:
        for d in listOfDataFramesPerClass[k]['gyro'][a]:
            keep_col = ['x','y','z']
            new_df = d[keep_col]
            new_df = new_df[:-24]
            trimmed_df = new_df.iloc[24:]
            if len(trimmed_df) < 50:
                print(str(ctr) + " " + str(len(trimmed_df)))
            trimmedDataFramesPerClass[a].append(trimmed_df)
            ctr += 1
        
    print("Done Trimming " + str(ctr) + " files")
    dataFrameDictionary["gyro"] = trimmedDataFramesPerClass
    listOfTrimmedGyroDataFramesPerClass[k] = dataFrameDictionary
    
print("Done Trimming Gyro data!")

Normalization (For Acceleration and Gyroscope)

In [ ]:
from sklearn import preprocessing

listOfNormalizedAccDataFramesPerClass = {}

for k in listOfTrimmedAccDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    NormalizedDataFramesPerClass = {}
    NormalizedDataFramesPerClass['left'] = []
    NormalizedDataFramesPerClass['right'] = []
    dataFrameDictionary = {}

    for a in listOfDataFramesPerClass[k]['gyro']:
        for d in listOfTrimmedAccDataFramesPerClass[k]['acc'][a]:
            #print(str(len(d.index)) + " " + str(ctr))
            # Create x, where x the 'scores' column's values as floats
            x = d.values.astype(float)

            # Create a minimum and maximum processor object
            min_max_scaler = preprocessing.MinMaxScaler()

            # Create an object to transform the data to fit minmax processor
            x_scaled = min_max_scaler.fit_transform(x)

            # Run the normalizer on the dataframe
            df_normalized = pd.DataFrame(x_scaled)
            #df_normalized
            NormalizedDataFramesPerClass[a].append(df_normalized)
            ctr += 1
        
    print("Done Normalizing " + str(ctr) + " files")
    dataFrameDictionary["acc"] = NormalizedDataFramesPerClass
    listOfNormalizedAccDataFramesPerClass[k] = dataFrameDictionary  
    
print("Acc Data Normalization done!!")



In [ ]:
from sklearn import preprocessing

listOfNormalizedGyroDataFramesPerClass = {}

for k in listOfTrimmedGyroDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    NormalizedDataFramesPerClass = {}
    NormalizedDataFramesPerClass['left'] = []
    NormalizedDataFramesPerClass['right'] = []
    dataFrameDictionary = {}

    for a in listOfDataFramesPerClass[k]['gyro']:
        for d in listOfTrimmedGyroDataFramesPerClass[k]['gyro'][a]:
            #print(str(len(d.index)) + " " + str(ctr))
            # Create x, where x the 'scores' column's values as floats
            x = d.values.astype(float)

            # Create a minimum and maximum processor object
            min_max_scaler = preprocessing.MinMaxScaler()

            # Create an object to transform the data to fit minmax processor
            x_scaled = min_max_scaler.fit_transform(x)

            # Run the normalizer on the dataframe
            df_normalized = pd.DataFrame(x_scaled)
            #df_normalized
            NormalizedDataFramesPerClass[a].append(df_normalized)
            ctr += 1
        
    print("Done Normalizing " + str(ctr) + " files")
    dataFrameDictionary["gyro"] = NormalizedDataFramesPerClass
    listOfNormalizedGyroDataFramesPerClass[k] = dataFrameDictionary  
    
print("Gyro Data Normalization done!!")



Compute Features

In [ ]:
import pandas as pd
from statsmodels import robust
import numpy as np
import scipy as sp

#kurtosis
#standarddev
#mean
#variance
#entropy
#meanabsval
#rootmeansquare
#Signal Magnitude Area
#Integration

listOfAccVarianceDataFramePerClass = {}
listOfAccVarianceDataFramePerClass['right'] = []
listOfAccVarianceDataFramePerClass['left'] = []

listOfAccMavDataFramePerClass = {}
listOfAccMavDataFramePerClass['right'] = []
listOfAccMavDataFramePerClass['left'] = []

listOfAccMeanDataFramePerClass = {}
listOfAccMeanDataFramePerClass['right'] = []
listOfAccMeanDataFramePerClass['left'] = []

listOfAccSdDataFramePerClass = {}
listOfAccSdDataFramePerClass['right'] = []
listOfAccSdDataFramePerClass['left'] = []

listOfAccRmsDataFramePerClass = {}
listOfAccRmsDataFramePerClass['right'] = []
listOfAccRmsDataFramePerClass['left'] = []

listOfAccEntropyDataFramePerClass = {}
listOfAccEntropyDataFramePerClass['right'] = []
listOfAccEntropyDataFramePerClass['left'] = []

listOfAccKurtosisDataFramePerClass = {}
listOfAccKurtosisDataFramePerClass['right'] = []
listOfAccKurtosisDataFramePerClass['left'] = []

listOfAccSkewnessDataFramePerClass = {}
listOfAccSkewnessDataFramePerClass['right'] = []
listOfAccSkewnessDataFramePerClass['left'] = []

listOfAccSmaDataFramePerClass = {}
listOfAccSmaDataFramePerClass['right'] = []
listOfAccSmaDataFramePerClass['left'] = []

listOfAccIntegrationDataFramePerClass = {}
listOfAccIntegrationDataFramePerClass['right'] = []
listOfAccIntegrationDataFramePerClass['left'] = []

for k in listOfNormalizedAccDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    
    for a in listOfNormalizedAccDataFramesPerClass[k]['acc']:
        ctr = 0
        arm = a + " "

        varianceDataFrame = pd.DataFrame(columns = [(arm+ 'Variance of ACC x'), (arm+ 'Variance of ACC y'), (arm+ 'Variance of ACC z')])
        mavDataFrame = pd.DataFrame(columns = [(arm + 'Mav of ACC x'), (arm + 'Mav of ACC y'), (arm + 'Mav of ACC z')])
        meanDataFrame = pd.DataFrame(columns = [(arm + 'Mean of ACC x'), (arm + 'Mean of ACC y'), (arm + 'Mean of ACC z')])
        sdDataFrame = pd.DataFrame(columns = [(arm + 'Sd of ACC x'), (arm + 'Sd of ACC y'), (arm + 'Sd of ACC z')])
        rmsDataFrame = pd.DataFrame(columns = [(arm + 'Rms of ACC x'), (arm + 'Rms of ACC y'), (arm + 'Rms of ACC z')])
        entropyDataFrame = pd.DataFrame(columns = [(arm + 'Entropy of ACC x'), (arm + 'Entropy of ACC y'), (arm + 'Entropy of ACC z')])
        kurtosisDataFrame = pd.DataFrame(columns = [(arm + 'Kurtosis of ACC x'), (arm + 'Kurtosis of ACC y'), (arm + 'Kurtosis of ACC z')])
        skewnessDataFrame = pd.DataFrame(columns = [(arm + 'Skewness of ACC x'), (arm + 'Skewness of ACC y'), (arm + 'Skewness of ACC z')])
        smaDataFrame = pd.DataFrame(columns = [(arm + 'SMA of ACC')])
        integrationDataFrame = pd.DataFrame(columns = [(arm + 'Integration of ACC x'), (arm + 'Integration of ACC y'), (arm + 'Integration of ACC z')])

        
        for df_normalized in listOfNormalizedAccDataFramesPerClass[k]['acc'][a]:
            varianceDataFrame.loc[ctr] = [df_normalized.var()[0], df_normalized.var()[1], df_normalized.var()[2]]
            mavDataFrame.loc[ctr] = [robust.mad(df_normalized)[0], robust.mad(df_normalized)[1], robust.mad(df_normalized)[2]]
            meanDataFrame.loc[ctr] = [np.mean(df_normalized)[0], np.mean(df_normalized)[1], np.mean(df_normalized)[2]]
            sdDataFrame.loc[ctr] = [np.std(df_normalized)[0], np.std(df_normalized)[1], np.std(df_normalized)[2]]
            rmsDataFrame.loc[ctr] = [np.sqrt(np.mean(df_normalized[0].values**2)), np.sqrt(np.mean(df_normalized[1].values**2)), np.sqrt(np.mean(df_normalized[2].values**2))]
            entropyDataFrame.loc[ctr] = [sp.stats.entropy(df_normalized)[0], sp.stats.entropy(df_normalized)[1], sp.stats.entropy(df_normalized)[2]]
            kurtosisDataFrame.loc[ctr] = [df_normalized.kurtosis()[0], df_normalized.kurtosis()[1], df_normalized.kurtosis()[2]]
            skewnessDataFrame.loc[ctr] = [df_normalized.skew(axis=0)[0], df_normalized.skew(axis=0)[1], df_normalized.skew(axis=0)[2]]
            length = df_normalized.shape[0]
            sma = 0
            for x in range(0, length):
                # .iloc[x,0] is accessing row, column
                sma += abs(df_normalized.iloc[x,0]) + df_normalized.iloc[x,1] + df_normalized.iloc[x,2]
            sma = sma / length
            smaDataFrame.loc[ctr] = [sma]
            integrationDataFrame.loc[ctr] = [sp.integrate.simps(df_normalized, axis=0)[0],sp.integrate.simps(df_normalized, axis=0)[1], sp.integrate.simps(df_normalized, axis=0)[2]]

            ctr +=1 
            
        print(len(varianceDataFrame))
        listOfAccVarianceDataFramePerClass[a].append(varianceDataFrame)
        listOfAccMavDataFramePerClass[a].append(mavDataFrame)
        listOfAccMeanDataFramePerClass[a].append(meanDataFrame)
        listOfAccSdDataFramePerClass[a].append(sdDataFrame)
        listOfAccRmsDataFramePerClass[a].append(rmsDataFrame)
        listOfAccEntropyDataFramePerClass[a].append(entropyDataFrame) 
        listOfAccKurtosisDataFramePerClass[a].append(kurtosisDataFrame) 
        listOfAccSkewnessDataFramePerClass[a].append(skewnessDataFrame) 
        listOfAccSmaDataFramePerClass[a].append(smaDataFrame) 
        listOfAccIntegrationDataFramePerClass[a].append(integrationDataFrame) 
        
        print("Done computing acc features " + str(ctr) + a + " arm files")

print("Done computing acc features!!!!!!!")

In [ ]:
import pandas as pd
from statsmodels import robust
import numpy as np
import scipy as sp

#kurtosis
#standarddev
#mean
#variance
#entropy
#meanabsval
#rootmeansquare
#Signal Magnitude Area
#Integration

listOfGyroVarianceDataFramePerClass = {}
listOfGyroVarianceDataFramePerClass['right'] = []
listOfGyroVarianceDataFramePerClass['left'] = []

listOfGyroMavDataFramePerClass = {}
listOfGyroMavDataFramePerClass['right'] = []
listOfGyroMavDataFramePerClass['left'] = []

listOfGyroMeanDataFramePerClass = {}
listOfGyroMeanDataFramePerClass['right'] = []
listOfGyroMeanDataFramePerClass['left'] = []

listOfGyroSdDataFramePerClass = {}
listOfGyroSdDataFramePerClass['right'] = []
listOfGyroSdDataFramePerClass['left'] = []

listOfGyroRmsDataFramePerClass = {}
listOfGyroRmsDataFramePerClass['right'] = []
listOfGyroRmsDataFramePerClass['left'] = []

listOfGyroEntropyDataFramePerClass = {}
listOfGyroEntropyDataFramePerClass['right'] = []
listOfGyroEntropyDataFramePerClass['left'] = []

listOfGyroKurtosisDataFramePerClass = {}
listOfGyroKurtosisDataFramePerClass['right'] = []
listOfGyroKurtosisDataFramePerClass['left'] = []

listOfGyroSkewnessDataFramePerClass = {}
listOfGyroSkewnessDataFramePerClass['right'] = []
listOfGyroSkewnessDataFramePerClass['left'] = []

listOfGyroSmaDataFramePerClass = {}
listOfGyroSmaDataFramePerClass['right'] = []
listOfGyroSmaDataFramePerClass['left'] = []

listOfGyroIntegrationDataFramePerClass = {}
listOfGyroIntegrationDataFramePerClass['right'] = []
listOfGyroIntegrationDataFramePerClass['left'] = []

for k in listOfNormalizedGyroDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    
    for a in listOfNormalizedGyroDataFramesPerClass[k]['gyro']:
        ctr = 0
        arm = a + " "
        varianceDataFrame = pd.DataFrame(columns = [(arm + 'Variance of Gyro x'), (arm+ 'Variance of Gyro y'), (arm+ 'Variance of Gyro z')])
        mavDataFrame = pd.DataFrame(columns = [(arm+ 'Mav of Gyro x'), (arm+ 'Mav of Gyro y'), (arm+ 'Mav of Gyro z')])
        meanDataFrame = pd.DataFrame(columns = [(arm+ 'Mean of Gyro x'), (arm+ 'Mean of Gyro y'), (arm+ 'Mean of Gyro z')])
        sdDataFrame = pd.DataFrame(columns = [(arm+ 'Sd of Gyro x'), (arm+ 'Sd of Gyro y'), (arm+ 'Sd of Gyro z')])
        rmsDataFrame = pd.DataFrame(columns = [(arm+ 'Rms of Gyro x'), (arm+ 'Rms of Gyro y'), (arm+ 'Rms of Gyro z')])
        entropyDataFrame = pd.DataFrame(columns = [(arm+ 'Entropy of Gyro x'), (arm+ 'Entropy of Gyro y'), (arm+ 'Entropy of Gyro z')])
        kurtosisDataFrame = pd.DataFrame(columns = [(arm+ 'Kurtosis of Gyro x'), (arm+ 'Kurtosis of Gyro y'), (arm+ 'Kurtosis of Gyro z')])
        skewnessDataFrame = pd.DataFrame(columns = [(arm+ 'Skewness of Gyro x'), (arm+ 'Skewness of Gyro y'), (arm+ 'Skewness of Gyro z')])
        smaDataFrame = pd.DataFrame(columns = [(arm+ 'SMA of Gyro')])
        integrationDataFrame = pd.DataFrame(columns = [(arm+ 'Integration of Gyro x'), (arm+ 'Integration of Gyro y'), (arm+ 'Integration of Gyro z')])

        
        for df_normalized in listOfNormalizedGyroDataFramesPerClass[k]['gyro'][a]:
            varianceDataFrame.loc[ctr] = [df_normalized.var()[0], df_normalized.var()[1], df_normalized.var()[2]]
            mavDataFrame.loc[ctr] = [robust.mad(df_normalized)[0], robust.mad(df_normalized)[1], robust.mad(df_normalized)[2]]
            meanDataFrame.loc[ctr] = [np.mean(df_normalized)[0], np.mean(df_normalized)[1], np.mean(df_normalized)[2]]
            sdDataFrame.loc[ctr] = [np.std(df_normalized)[0], np.std(df_normalized)[1], np.std(df_normalized)[2]]
            rmsDataFrame.loc[ctr] = [np.sqrt(np.mean(df_normalized[0].values**2)), np.sqrt(np.mean(df_normalized[1].values**2)), np.sqrt(np.mean(df_normalized[2].values**2))]
            entropyDataFrame.loc[ctr] = [sp.stats.entropy(df_normalized)[0], sp.stats.entropy(df_normalized)[1], sp.stats.entropy(df_normalized)[2],]
            kurtosisDataFrame.loc[ctr] = [df_normalized.kurtosis()[0], df_normalized.kurtosis()[1], df_normalized.kurtosis()[2]]
            skewnessDataFrame.loc[ctr] = [df_normalized.skew(axis=0)[0], df_normalized.skew(axis=0)[1], df_normalized.skew(axis=0)[2]]
            length = df_normalized.shape[0]
            sma = 0
            for x in range(0, length):
                # .iloc[x,0] is Gyroessing row, column
                sma += abs(df_normalized.iloc[x,0]) + df_normalized.iloc[x,1] + df_normalized.iloc[x,2]
            sma = sma / length
            smaDataFrame.loc[ctr] = [sma]
            integrationDataFrame.loc[ctr] = [sp.integrate.simps(df_normalized, axis=0)[0],sp.integrate.simps(df_normalized, axis=0)[1], sp.integrate.simps(df_normalized, axis=0)[2]]

            ctr +=1 
            
        print(len(varianceDataFrame))
        listOfGyroVarianceDataFramePerClass[a].append(varianceDataFrame)
        listOfGyroMavDataFramePerClass[a].append(mavDataFrame)
        listOfGyroMeanDataFramePerClass[a].append(meanDataFrame)
        listOfGyroSdDataFramePerClass[a].append(sdDataFrame)
        listOfGyroRmsDataFramePerClass[a].append(rmsDataFrame)
        listOfGyroEntropyDataFramePerClass[a].append(entropyDataFrame) 
        listOfGyroKurtosisDataFramePerClass[a].append(kurtosisDataFrame) 
        listOfGyroSkewnessDataFramePerClass[a].append(skewnessDataFrame) 
        listOfGyroSmaDataFramePerClass[a].append(smaDataFrame) 
        listOfGyroIntegrationDataFramePerClass[a].append(integrationDataFrame) 
        
        print("Done computing Gyro features " + str(ctr) + a + " arm files")
        
print("Done computing gyro features!!!!!!!")

Label Class

In [ ]:
#print(listOfMavDataFramePerClass[0])
#print(listOfRmsDataFramePerClass[0])
ctr = 0

listOfLabeledAccDataPerClass = {}
listOfLabeledAccDataPerClass['right'] = {}
listOfLabeledAccDataPerClass['left'] = {}


for a in listOfNormalizedAccDataFramesPerClass[k]['acc']:

    accDataFramesPerFeature = {}
    ctr = 0
    for k in listOfTrimmedAccDataFramesPerClass.keys():
        accDataFramesPerFeature['variance'] = listOfAccVarianceDataFramePerClass[a][ctr]
        accDataFramesPerFeature['mav'] = listOfAccMavDataFramePerClass[a][ctr]
        accDataFramesPerFeature['rms'] = listOfAccRmsDataFramePerClass[a][ctr]
        accDataFramesPerFeature['sd'] = listOfAccSdDataFramePerClass[a][ctr]
        accDataFramesPerFeature['mean'] = listOfAccMeanDataFramePerClass[a][ctr]
        accDataFramesPerFeature['entropy'] = listOfAccEntropyDataFramePerClass[a][ctr]
        accDataFramesPerFeature['kurtosis'] = listOfAccKurtosisDataFramePerClass[a][ctr]
        accDataFramesPerFeature['skewness'] = listOfAccSkewnessDataFramePerClass[a][ctr]
        accDataFramesPerFeature['sma'] = listOfAccSmaDataFramePerClass[a][ctr]
        accDataFramesPerFeature['integration'] = listOfAccIntegrationDataFramePerClass[a][ctr]

        for k2 in accDataFramesPerFeature.keys():
            if k2 != 'variance':
                for i in accDataFramesPerFeature[k2].columns:
                    accDataFramesPerFeature['variance'][i] = accDataFramesPerFeature[k2][i]
        
        accDataFramesPerFeature['variance']['Class'] = k
        listOfLabeledAccDataPerClass[a][k] = accDataFramesPerFeature['variance']
        print("Done labeling acc data of " + k +" "+ a + " arm files")
        ctr += 1

In [ ]:
#print(listOfMavDataFramePerClass[0])
#print(listOfRmsDataFramePerClass[0])
ctr = 0

listOfLabeledGyroDataPerClass = {}
listOfLabeledGyroDataPerClass['right'] = {}
listOfLabeledGyroDataPerClass['left'] = {}


for a in listOfNormalizedGyroDataFramesPerClass[k]['gyro']:

    GyroDataFramesPerFeature = {}
    ctr = 0
    for k in listOfTrimmedGyroDataFramesPerClass.keys():
        GyroDataFramesPerFeature['variance'] = listOfGyroVarianceDataFramePerClass[a][ctr]
        GyroDataFramesPerFeature['mav'] = listOfGyroMavDataFramePerClass[a][ctr]
        GyroDataFramesPerFeature['rms'] = listOfGyroRmsDataFramePerClass[a][ctr]
        GyroDataFramesPerFeature['sd'] = listOfGyroSdDataFramePerClass[a][ctr]
        GyroDataFramesPerFeature['mean'] = listOfGyroMeanDataFramePerClass[a][ctr]
        GyroDataFramesPerFeature['entropy'] = listOfGyroEntropyDataFramePerClass[a][ctr]
        GyroDataFramesPerFeature['kurtosis'] = listOfGyroKurtosisDataFramePerClass[a][ctr]
        GyroDataFramesPerFeature['skewness'] = listOfGyroSkewnessDataFramePerClass[a][ctr]
        GyroDataFramesPerFeature['sma'] = listOfGyroSmaDataFramePerClass[a][ctr]
        GyroDataFramesPerFeature['integration'] = listOfGyroIntegrationDataFramePerClass[a][ctr]

        for k2 in GyroDataFramesPerFeature.keys():
            if k2 != 'variance':
                for i in GyroDataFramesPerFeature[k2].columns:
                    GyroDataFramesPerFeature['variance'][i] = GyroDataFramesPerFeature[k2][i]
        
        GyroDataFramesPerFeature['variance']['Class'] = k
        listOfLabeledGyroDataPerClass[a][k] = GyroDataFramesPerFeature['variance']
        print("Done labeling Gyro data of " + k +" "+ a + " arm files")
        ctr += 1

In [ ]:
test = listOfLabeledGyroDataPerClass['left']['ano']
testPerClass = {}
for k in listOfTrimmedGyroDataFramesPerClass.keys():
    #print(('%s %s') % (a, k))
    testPerClass[k] = []
    test = listOfLabeledGyroDataPerClass['left'][k]
    for i in listOfLabeledGyroDataPerClass['right'][k]:
        test[i] = listOfLabeledGyroDataPerClass['right'][k][i]
    for i in listOfLabeledAccDataPerClass['left'][k].columns:
        test[i] = listOfLabeledAccDataPerClass['left'][k][i]
    for i in listOfLabeledAccDataPerClass['right'][k].columns:
        test[i] = listOfLabeledAccDataPerClass['right'][k][i]
    for i in listOfLabeledDataPerClass['left'][k].columns:
        test[i] = listOfLabeledDataPerClass['left'][k][i]
    for i in listOfLabeledDataPerClass['right'][k].columns:
        test[i] = listOfLabeledDataPerClass['right'][k][i]                        #emg data
    testPerClass[k] = test
# test
# test.to_csv('/Users/user/Desktop/test.csv', encoding='utf-8', index=False)
newDf = testPerClass['ano']
for k in listOfTrimmedAccDataFramesPerClass.keys():
    if k != 'ano':
        print(k)
        newDf= newDf.append(testPerClass[k])
    
    
newDf.to_csv('/Users/user/Desktop/ConsolidatedData.csv', encoding='utf-8', index=False)


In [ ]:
# for a in listOfNormalizedAccDataFramesPerClass[k]['acc']:
    


# for k2 in GyroDataFramesPerFeature.keys():
#     for i in GyroDataFramesPerFeature[k2].columns:
#         dataFramesPerFeature['variance'][i] = GyroDataFramesPerFeature[k2][i]
            
# for k2 in accDataFramesPerFeature.keys():
#     for i in accDataFramesPerFeature[k2].columns:
#         dataFramesPerFeature['variance'][i] = accDataFramesPerFeature[k2][i]

# for a in listOfNormalizedAccDataFramesPerClass[k]['acc']:
#     newDf = listOfLabeledAccDataPerClass[a]['ano']
#     for k in listOfTrimmedAccDataFramesPerClass.keys():
#         if k != 'ano':
#             print(k)
#             newDf= newDf.append(listOfLabeledAccDataPerClass[a][k])
# dataFramesPerFeature.to_csv('/Users/user/Desktop/test.csv', encoding='utf-8', index=False)

Data Consolidation (save to csv)

In [ ]:
# for a in listOfNormalizedAccDataFramesPerClass[k]['acc']:
#     newDf = listOfLabeledAccDataPerClass[a]['ano']
#     for k in listOfTrimmedAccDataFramesPerClass.keys():
#         if k != 'ano':
#             print(k)
#             newDf= newDf.append(listOfLabeledAccDataPerClass[a][k])
    
    
#     newDf.to_csv('/Users/user/Desktop/'+ a +'ArmConsolidatedAcc.csv', encoding='utf-8', index=False)

In [ ]:
# for a in listOfNormalizedGyroDataFramesPerClass[k]['gyro']:
#     newDf = listOfLabeledGyroDataPerClass[a]['ano']
#     for k in listOfTrimmedGyroDataFramesPerClass.keys():
#         if k != 'ano':
#             print(k)
#             newDf= newDf.append(listOfLabeledGyroDataPerClass[a][k])
    
    
#     newDf.to_csv('/Users/user/Desktop/'+ a +'ArmConsolidatedGyro.csv', encoding='utf-8', index=False)